# OpenStreetMap Data Data Audit
___

Matthew Flood

> This notebook contains the details of the Audit.  Please see [README.md](README.md), which is my project submission document.

### Map Area
 New Orleans, LA, United States

- [https://www.openstreetmap.org/relation/131885](https://www.openstreetmap.org/relation/131885)
- [https://www.openstreetmap.org/export#map=11/30.0326/-89.8826](https://www.openstreetmap.org/export#map=11/30.0326/-89.8826)
- [https://overpass-api.de/api/map?bbox=-90.2170,29.8633,-89.5482,30.2015](https://overpass-api.de/api/map?bbox=-90.2170,29.8633,-89.5482,30.2015)


In [1]:
import re
import notebook_audit_methods

MAP_PATH="downloaded_maps/new_orleans_city.osm"


## Audit the top level tags

> First, we want to identify all the top level tags in the data set to make sure the document looks sane from a high level.

In [2]:
notebook_audit_methods.identify_tags_fullpath(MAP_PATH)

osm                           1
osm/note                      1
osm/meta                      1
osm/bounds                    1
osm/node                1786106
osm/node/tag             130282
osm/way                  189487
osm/way/nd              2050430
osm/way/tag              535711
osm/relation               1407
osm/relation/member       25395
osm/relation/tag           7114
osm/remark                    1


> This structure is what we expected and nothing looks out of place.  We see &lt;node&gt; and &lt;way&gt; tags.  Those both have nested &lt;tag&gt; elements, and &lt;way&gt; tags also have nested &lt;nd&gt; tags.

## Audit the tag 'keys' 

> Now we want to audit the "k" value for each "&lt;tag&gt;" and see if there are any potential problems.  The following utility function will put each key into one of four buckets.

        1. all lowercase and valid
        2. all lowercase with a ':' in the middle, and valid
        3. ones with bad characters
        4. others (none of the above)

> I want to look at Nodes and Ways separately, so I'm going to store "key" info for each of thoise in separate variables

In [3]:
node_lowercase_tags, node_lowercase_colon_tags, node_bad_char_tags, node_other_tags = notebook_audit_methods.audit_tag_keys(MAP_PATH, "node")
way_lowercase_tags, way_lowercase_colon_tags, way_bad_char_tags, way_other_tags = notebook_audit_methods.audit_tag_keys(MAP_PATH, "way")


### tags with bad characters
First, let's look a the keys with bad characters in nodes, and then ways

In [4]:
sorted_keys = list(node_bad_char_tags.keys())
sorted_keys.sort()
for item in sorted_keys:
    value_set = node_bad_char_tags[item]
    print("{0: <20} {1: >10} {2:}".format(item, len(value_set), ", ".join(list(value_set))[:80] ) )

ADDRESS_LA                   85 9028 EDINBURGH ST, 9016 OLEANDER ST, 9334 COLAPISSA ST, 3223 CHERRY ST, 3016 HAM
FIXME                         9 Find out the English name and exact location, verify, can be amenity=restaurant,
HIV                           1 Delgado STD Clinic
Payments                      1 MasterCard, Amex, Cash, Visa, Discover, Checks


In [5]:
sorted_keys = list(way_bad_char_tags.keys())
sorted_keys.sort()
for item in sorted_keys:
    value_set = way_bad_char_tags[item]
    print("{0: <20} {1: >10} {2:}".format(item, len(value_set), ", ".join(list(value_set))[:80] ) )

ADDRESS_LA                   13 9308 OLIVE ST, 3212 HAMILTON ST, 3204 HAMILTON ST, 9200 FORSHEY ST, 9216 OLEANDE
FIXME                         6 dual carriageway, check lanes, verify, route temporarily suspended - 12/8/2009, 
FIXME:ncn_ref                 1 verify
FIXME:ref                     1 verify north end
NHD:ComID                  6845 139149733, 139202458, 139203845, 139145352, 139182494, 148744716, 139187755, 139
NHD:Elevation                 5 0.60000000000, -2.70000000000, 0.90000000000, -2.40000000000, -3.00000000000
NHD:FCode                    13 46006, 39009, 36900, 39004, 33600, 39001, 39800, 43612, 46600, 43609, 44500, 460
NHD:FDate                    12 2005/01/14, 2006/09/22, 2009/01/15, 2006/05/31, 2005/10/29, 2005/12/07, 2005/08/
NHD:FTYPE                     7 Lock Chamber, StreamRiver, Reservoir, SeaOcean, SwampMarsh, CanalDitch, LakePond
NHD:FType                     5 StreamRiver, Wall, CanalDitch, Lock Chamber, Gate
NHD:GNIS_ID                  12 00554556, 0

> So, these keys do not really have bad data, they just have capital letters.  And some, like `NHD:FTYPE` and `NHD:FType` have different capitlization.
> We will address these by forcing all the k values to lowercase when we perform our import


### Other keys

> Now we want to look at the "other" bucket to see what kind of issues there are.

In [6]:
sorted_keys = list(node_other_tags.keys())
sorted_keys.sort()
for item in sorted_keys:
    value_set = node_other_tags[item]
    print("{0: <20} {1: >10} {2:}".format(item, len(value_set), ", ".join(list(value_set))[:80] ) )

amenity_1                     1 cafe
currency:ETH                  1 yes
currency:LTC                  1 yes
currency:XBT                  1 yes
generator:output:electricity          1 560 MW
gnis:Class                    1 Populated Place
gnis:County                   5 Jefferson, Plaquemines, St. Tammany, Orleans, St. Bernard
gnis:County_num               5 103, 051, 075, 071, 087
gnis:ST_alpha                 1 LA
gnis:ST_num                   1 22
name:etymology:wikidata          1 Q8027
name_1                        1 Lakefront Airport
service:bicycle:diy           1 yes
service:bicycle:pump          1 yes
service:bicycle:rental          1 yes
service:bicycle:repair          1 yes
service:bicycle:retail          1 yes
service:bicycle:second_hand          1 yes
service:bicycle:tools          1 yes
socket:nema_5_20              1 2
socket:type1                  2 1, 2
source:name:oc                1 Lo Congrès
wikipedia_1                   1 pl:Pomnik Andrew Jacksona w Waszyngtonie


In [7]:
sorted_keys = list(way_other_tags.keys())
sorted_keys.sort()
for item in sorted_keys:
    value_set = way_other_tags[item]
    print("{0: <20} {1: >10} {2:}".format(item, len(value_set), ", ".join(list(value_set))[:80] ) )

building:roof:material          1 glass
cycleway:left:oneway          1 no
cycleway:left:width           1 9'
cycleway:right:oneway          1 no
cycleway:right:width          5 12', 8', 9, 9', 11'
destination:ref:lanes          2 LA 45, LA 45|LA 45
gnis:Class                    1 Populated Place
gnis:County                   1 Jefferson
gnis:County_num               1 051
gnis:ST_alpha                 1 LA
gnis:ST_num                   1 22
name:etymology:wikidata          1 Q8027
name_1                       93 Selma Avenue, South I- 10 Service Road West, Beech Drive, Belmont Place, Tracy D
name_2                        7 Louisiana Hwy 406, South Causeway Boulevard, Sanctuary Drive, West Saint Bernard
plant:output:electricity          2 1646 MW, 959 MW
ref:AEFE                      1 272N15
service:vehicle:body_repair          1 yes
sidewalk:both:surface          1 concrete
tiger:name_base_1           119 Medical Center, State Route 611-6, Cameron, Belmont, State, Decomine, General C

> Again, we see issues related to capitalization.  We also see some numbers, which have to do with the fact that a key can only be used once for a tag, so if you want more than one name, then instead of having three tags with `name`, you need to have `name`, `name_1` and `name_2`.

> To verify that this is the case, I want to look at a few tags that have `name_` or `amenity_`

#### Look into name_1 and amenity_1

>This function will find entities that have a tag with the 
given `tag_key` (e.g. **name_1**) and print out the full set of tags for the node/way/relation

In [8]:
notebook_audit_methods.print_example_entities(MAP_PATH, "name_1", max_results=3)

Example node
{'addr:state': {'LA'},
 'aeroway': {'aerodrome'},
 'closest_town': {'New Orleans, Louisiana'},
 'ele': {'1'},
 'faa': {'NEW'},
 'gnis:county_name': {'Orleans'},
 'gnis:created': {'08/01/1991'},
 'gnis:feature_id': {'1632892'},
 'gnis:feature_type': {'Airport'},
 'iata': {'NEW'},
 'icao': {'KNEW'},
 'name': {'New Orleans Lakefront Airport'},
 'name:en': {'New Orleans Lakefront Airport'},
 'name_1': {'Lakefront Airport'},
 'operator': {'Orleans Levee District'},
 'source': {'wikipedia'},
 'source_ref': {'geonames.usgs.gov'},
 'type': {'public'},
 'wikidata': {'Q10853543'},
 'wikipedia': {'en:New Orleans Lakefront Airport'}}
Example way
{'highway': {'secondary'},
 'maxspeed': {'45 mph'},
 'name': {'Peters Road'},
 'name_1': {'State Route 3017'},
 'ref': {'LA 3017'},
 'tiger:cfcc': {'A41;A35'},
 'tiger:county': {'Jefferson, LA'},
 'tiger:name_base': {'Peters'},
 'tiger:name_base_1': {'State Route 3017'},
 'tiger:name_type': {'Rd'},
 'tiger:reviewed': {'no'},
 'tiger:zip_left':

> Yup - these all have name_1 as alternates to the name key. I want to look at amenity_1 as well

In [9]:
notebook_audit_methods.print_example_entities(MAP_PATH, "amenity_1", max_results=3)

Example node
{'addr:city': {'New Orleans'},
 'addr:housenumber': {'2372 #130'},
 'addr:postcode': {'70117'},
 'addr:state': {'LA'},
 'addr:street': {'Saint Claude Avenue'},
 'amenity': {'restaurant'},
 'amenity_1': {'cafe'},
 'delivery': {'no'},
 'name': {'The Spotted Cat'},
 'takeaway': {'yes'}}


### Lowercase keys without semicolons 

> Now we want to look at the lowercase keys, the ones without semicolons.

In [10]:
# First look at lowercase keys for ways
sorted_keys = list(way_lowercase_tags.keys())
sorted_keys.sort()
for item in sorted_keys:
    value_set = way_lowercase_tags[item]
    print("{0: <20} {1: >10} {2:}".format(item, len(value_set), ", ".join(list(value_set))[:80] ) )

access                        7 destination, yes, permissive, private, no, delivery, customers
admin_level                   2 8, 6
aeroway                       6 helipad, taxiway, runway, apron, terminal, hangar
alt_name                     15 Alcee Fortier Boulevard, Highway 433, MRGO, Goldring/Woldenberg Great Lawn, Nune
amenity                      48 cafe, fire_station, fountain, police, fast_food, dive_centre, library, cinema, p
architect                     4 Emile Weil, Edward Durell Stone, Charles Moore, DMJM
area                          2 yes, no
artist_name                   1 Multiple
artwork_type                  2 sculpture, statue
atm                           1 yes
attribution                   1 NHD
backrest                      1 yes
barrier                       9 wall, dyke, retaining_wall, gate, hedge, guard_rail, wire_fence, kerb, fence
bicycle                       4 designated, permissive, yes, no
boat                          2 yes, no
border_type            

In [11]:
# Now look at lower case keys for nodes
sorted_keys = list(node_lowercase_tags.keys())
sorted_keys.sort()
for item in sorted_keys:
    value_set = node_lowercase_tags[item]
    print("{0: <20} {1: >10} {2:}".format(item, len(value_set), ", ".join(list(value_set))[:80] ) )

access                        5 yes, permissive, no, private, customers
aeroway                       3 helipad, holding_position, aerodrome
alt_name                     19 NOLA MERCHANDISE, Tastee Donuts, Optical Shop of New Orleans, Bienville monument
amenity                      64 cafe, fire_station, Jazz Club, fountain, Emergeny Base Logistics, biergarten, ki
artist                        1 Clark Mills
artist_name                  14 Franco Alessandrini, Yaacov Agam, Madeleine Faust, Enrique Alferez, Terry Weldin
artwork_type                  4 sculpture, mural, statue, painting
atm                           2 yes, no
attribution                   1 USGS 2001 County Boundary
automated                     1 yes
backrest                      1 yes
barrier                       8 bollard, lift_gate, entrance, yes, block, gate, guard_rail, swing_gate
beauty                        3 nails, hair_removal, spa
bench                         2 yes, no
bicycle                       2 yes, no

> Nothing seems out of place there.  

# Lowercase keys with semicolons

Finally, we want to look at the ones with semicolons

In [12]:
sorted_keys = list(way_lowercase_colon_tags.keys())
sorted_keys.sort()
for item in sorted_keys:
    value_set = way_lowercase_colon_tags[item]
    print("{0: <20} {1: >10} {2:}".format(item, len(value_set), ", ".join(list(value_set))[:80] ) )

abandoned:railway             1 rail
abandoned:tourism             1 theme_park
addr:city                    16 Marrero, Harahan, Gretna, Jefferson, Metairie, New Oreleans, Avondale, Belle Cha
addr:country                  2 UA, US
addr:full                     2 99 Westbank Expy, 6000 Bullard Ave
addr:housename                2 Bagneris Home, Arabella Station
addr:housenumber           8908 8503, 9956, 8602, 3441, 9017, 6191, 8937, 6093, 3774, 13191, 9980, 4859, 26277, 
addr:postcode                30 70003, 70032, 70117, 70002, 70124, 70130, 70094, 70126, 70112-2625, 70072, 70006
addr:state                    4 LA, Louisiana, lA, la
addr:street                1998 West Alba Street, Gaines Street, Saint Anthony Street, Quince Street, A P Tureau
addr:unit                     1 115
amenity:bar                   1 yes
amenity:historic              1 hospital
brand:wikidata               32 Q2735242, Q154950, Q10397787, Q846301, Q483551, Q38076, Q37158, Q1047239, Q19604
brand:wikipedia   

In [13]:
sorted_keys = list(node_lowercase_colon_tags.keys())
sorted_keys.sort()
for item in sorted_keys:
    value_set = node_lowercase_colon_tags[item]
    print("{0: <20} {1: >10} {2:}".format(item, len(value_set), ", ".join(list(value_set))[:80] ) )

abandoned:man_made            1 lighthouse
addr:city                    17 Marrero, Terrytown, Chalmette, Harahan, Gretna, Metairie, New Oreleans, Harvey, 
addr:country                  1 US
addr:full                     6 1501 Manhattan Blvd, 4301 Chef Menteur Highway, 4810 Lapalco Blvd, 4001 Behrman 
addr:housename                9 Patio, Aframe, Chase Bank, Fritzel's - Traditional Jazz Club, Homestead, Malbrou
addr:housenumber           6685 8503, 8602, 14417, 3441, 9017, 8937, 3774, 3869, 4859, 7818, 5214, 7055, 5230, 1
addr:interpolation            2 200, 600
addr:place                    1 Suite 500
addr:postcode                37 70122, 70003, 70117, 70002, 70012, 70124, 70130, 70170, 70094, LA 70116, 70126, 
addr:state                    3 LA, Louisiana, La
addr:street                1294 Sail Street, Saint Anthony Street, Prentiss Avenue, Quince Street, A P Tureaud A
addr:unit                    31 Suite B, 110, #1825, Suite 1, Suite A-2, Suite 100, Suite 2, B, #A, 118, Suite 

### Auditing Street Names

> Now I want to look at street names to make sure they are consistent

> First we'll build a function that uses regexes to identiyfy "normal" street addresses.  We'll pass in an argument to flip between showing normal and irregular addresses.


In [14]:
value_counts = notebook_audit_methods.get_all_streets(MAP_PATH)

In [15]:
# print the values, along with the count of occurrences
sorted_keys = list(value_counts.keys())
sorted_keys.sort()
for item in sorted_keys:
    count = value_counts[item]
    print("{0: <20} {1: >10}".format(item, count) )


10th Street                  56
1227 Tulane Ave               1
12th Street                  33
14th Street                  49
16th Street                  40
18th Street                  51
1st Street                  366
20th Street                  38
22nd Street                  39
26th Street                  37
28th Street                  34
2nd Street                  368
30th Street                  37
3157 Gentilly Blvd #2019          1
32nd Street                  10
33rd Street                  17
34th Street                  21
35th Street                  13
36th Street                  17
37th Street                   9
38th Street                  16
39th Street                  17
3rd Street                  403
40th Street                  22
4th Street                  369
5th Street                    1
621 Canal Street              1
6th Street                  267
7th Street                  269
8th Street                  173
9th Street                   88
A P 

Wildair Drive                84
Wilkinson Street              7
Will Stutley Drive           22
William Street                2
Willow Drive                 19
Willow Ridge Drive            2
Willow Road                   2
Willow St.                    1
Willow Street               383
Willowbrae Drive             49
Willowbrook Drive            49
Wilshire Street              11
Wilson Avenue                71
Wilson Drive                 65
Wilton Drive                108
Wiltz Lane                   15
Wimbledon Court              18
Winchester Park Drive         48
Windsor Drive                 1
Windward Court               39
Wingate Drive                89
Winners Circle               21
Winrock Drive                27
Winston Street                1
Winthrop Place               18
Winthrop Street              12
Wirth Place                  33
Wisteria Street              95
Woodbine Drive               54
Woodland Avenue               1
Woodland Drive               43
Woodlan

In [16]:
street_type_re = re.compile(r'(.*) (\b\S+\.?)$', re.IGNORECASE)

# I built this list up as I looked at the results
expected = ["Street",
            "Avenue",
            "Court",
            "Drive",
            "Road",
            "Boulevard",
            "Place",
            "Lane",
            "Circle",
            "Cove",
            "Arcade",
            "Way",
            "Walk",
            "Highway",
            "Parkway",
            "Alley",
            "Plaza",
            "Trace"]

to_fix = {"St": "Street",
          "St.": "Street",
          "Ave": "Avenue",
          "Pkwy": "Parkway",
          "Hwy": "Highway",
          "Blvd": "Boulevard",
          "Dr": "Drive",
          "Ave.": "Avenue",
          "Pky": "Parkway"}

fullname_mapping = {"Banks": "Banks Street",
                    "Regent Street #C": "Regent Street",
                    "Dumaine": "Dumaine Street",
                    "Magazine Street;Magazine St": "Magazine Street",
                    "Magazine": "Magazine Street",
                    "Rosa PARK": "Rosa Park",
                    "Severn": "Severn Avenue",
                    "St. Peter": "Saint Peter Street",
                    "3157 Gentilly Blvd #2019": "Gentilly Boulevard",
                    "Marais": "Marais Street",
                    "Royal": "Royal Street",
                    "Canal Street at Bourbon": "Canal Street",
                    "General Taylor": "General Taylor Street",
                    "Gretna Blvd Ste A": "Gretna Boulevard",
                    "Manhattan Boulevard Building": "Manhattan Boulevard",
                    "Saint Charles Avenue, Suite 114-351": "Saint Charles Avenue",
                    "1227 Tulane Ave": "Tulane Avenue",
                    "621 Canal Street": "Canal Street",
                    "Westgrove PARK": "Westgrove Park",
                    "George 'Nick' Connor Drive": "George Nick Connor Drive",
                    "Bal of Square": "Banks Street",
                   }

sorted_keys = list(value_counts.keys())
sorted_keys.sort()

street_set = set()

for item in sorted_keys:
    
    # patch full items that are borked
    item = item.strip()
    item = fullname_mapping.get(item, item)
    
    if item.startswith("Magazine"):
        print(item)
    
    match = street_type_re.match(item)
    if match:
        #continue
        first_path = match.group(1)
        street_type = match.group(2)
        street_type = to_fix.get(street_type, street_type)
        
        street_name = "{} {}".format(first_path, street_type)
        
        beginnings = {'N ': 'North', 
                      'S ': 'South',
                      'E ': 'East',
                      'W ': 'West',
                      'St ': 'Saint',
                      'St. ': 'Saint'}
        for k, v in beginnings.items():
            start_string = "{} ".format(k)
            if street_name.startswith(k):
                street_name = street_name.replace(k, v, 1)

        # any St or St. that still remain are 'Saint'
        street_name.replace('St ', 'Saint ')
        street_name.replace('St. ', 'Saint ')
        
        street_set.add(street_name)
        
        
        #if street_type not in expected:
        #    print("{0: <20} '{1:}' '{2:}'".format(item, match.group(1), match.group(2)) )
        #else:
        #    print("Fixed Street: '{} {}'".format(first_path, street_type))
    else:
        street_set.add(street_name)
        #print("********************** Special {}".format(item))

        
for item in sorted(street_set):
    print("'{}'".format(item))
        

Magazine Street
Magazine St
Magazine Street
Magazine Street
'10th Street'
'12th Street'
'14th Street'
'16th Street'
'18th Street'
'1st Street'
'20th Street'
'22nd Street'
'26th Street'
'28th Street'
'2nd Street'
'30th Street'
'32nd Street'
'33rd Street'
'34th Street'
'35th Street'
'36th Street'
'37th Street'
'38th Street'
'39th Street'
'3rd Street'
'40th Street'
'4th Street'
'5th Street'
'6th Street'
'7th Street'
'8th Street'
'9th Street'
'A P Tureaud Avenue'
'Abalon Court'
'Abbey Drive'
'Aberdeen Road'
'Abundance Court'
'Abundance Street'
'Acacia Street'
'Acadiana Place'
'Ada Place'
'Adam Street'
'Adams Street'
'Adele Street'
'Admiralty Court'
'Adrian Street'
'Adventure Avenue'
'Afton Drive'
'Agate Street'
'Agriculture Place'
'Agriculture Street'
'Airline Drive'
'Airline Highway'
'Airway Street'
'Airwood Street'
'Alabama Street'
'Alabo Street'
'Alba Road'
'Albany Court'
'Albert Street'
'Alcee Fortier Boulevard'
'Alden Place'
'Alfred Street'
'Alhambra Street'
'Alice Court'
'Aline Stree

'West Roadway Street'
'West Robert E Lee Boulevard'
'West Rockton Circle'
'West Sandy Cove Drive'
'West St Andrews Circle'
'West St Roch Avenue'
'West Tamaron Boulevard'
'West Wheaton Circle'
'WestJudge Perez Drive'
'WestSt Bernard Highway'
'Westbank Expressway'
'Westbend Parkway'
'Westbrook Drive'
'Westchester Street'
'Western Street'
'Westgrove Park'
'Westhaven Road'
'Westlake Drive'
'Westpark Court'
'Westwood Drive'
'White Oak Avenue'
'Whiteside Road'
'Whitmore Place'
'Whitney Avenue'
'Wickfield Drive'
'Wildair Drive'
'Wilkinson Street'
'Will Stutley Drive'
'William Street'
'Willow Drive'
'Willow Ridge Drive'
'Willow Road'
'Willow Street'
'Willowbrae Drive'
'Willowbrook Drive'
'Wilshire Street'
'Wilson Avenue'
'Wilson Drive'
'Wilton Drive'
'Wiltz Lane'
'Wimbledon Court'
'Winchester Park Drive'
'Windsor Drive'
'Windward Court'
'Wingate Drive'
'Winners Circle'
'Winrock Drive'
'Winston Street'
'Winthrop Place'
'Winthrop Street'
'Wirth Place'
'Wisteria Street'
'Woodbine Drive'
'Woodland

There are a few streets that a still dubious:
    Bal Of Square
    Nearby: 401 4th Street
    'Paulina Drive' / 'Pauline Drive' / 'Pauline Street'
    'Pirate Alley' / 'Pirates Alley'
    'Ponce De Leon Street' / 'Ponce de Leon Street'
    'Robert E Lee Boulevard' / 'Robert E. Lee Boulevard'


In [17]:
notebook_audit_methods.print_entities_on_street(MAP_PATH, tag_key="addr:street", tag_value="Bal Of Square", max_results=10)

Example way
{'addr:housenumber': {'4910'},
 'addr:street': {'Bal Of Square'},
 'attribute: changeset': {'28548419'},
 'attribute: id': {'325925721'},
 'attribute: timestamp': {'2015-02-01T16:14:47Z'},
 'attribute: uid': {'22925'},
 'attribute: user': {'ELadner'},
 'attribute: version': {'1'},
 'building': {'yes'}}


> I want to get the lat and lon of that point, so I grep the file for that Way ID to see the ND elements.

```
BASH>grep 325925721 -A10 downloaded_maps/new_orleans_city.osm 
  <way id="325925721" version="1" timestamp="2015-02-01T16:14:47Z" changeset="28548419" uid="22925" user="ELadner">
    <nd ref="3325489118"/>
    <nd ref="3325489053"/>
    <nd ref="3325489149"/>
    <nd ref="3325489190"/>
    <nd ref="3325489203"/>
    <nd ref="3325489118"/>
    <tag k="addr:housenumber" v="4910"/>
    <tag k="addr:street" v="Bal Of Square"/>
    <tag k="building" v="yes"/>
  </way>
    ```

> And then grep for one of those ND refs to find a lat and lon

```
grep 3325489053 downloaded_maps/new_orleans_city.osm 
  <node id="3325489053" lat="29.9773393" lon="-90.1115287" version="1" timestamp="2015-02-01T16:14:16Z" changeset="28548419" uid="22925" user="ELadner"/>
```

> Next I take the lat and lon and perform search for that in google for:  29.9773393, -90.1115287

> The search results show a building at that location, and the associated address is:

```
 4910 Banks St, New Orleans, LA 70119
```

So, it looks like the original address was a typo.  I'm going to correct "Bal of Square" to be "Banks Street".
